# NB_SVM_LR_SENTIMENTANALYSIS_IMDB_BOW_RS

## Model 1:
1. UseFusing Naive Bayes and SVM/LR
2. Do Sentiment Analysis (SA)
3. Use Bag of Words (BOW)
4. Refined Stopwords (RS)

The notebook will have the following structure:
1. Importing libraries and data
2. Exploratory data analysis
3. Tokenization and training/validation split
4. Naive Bayes with bag-of-words
5. Naive Bayes with tf-idf
6. Logistic Regression/SVC with NB features
7. Inference and submission



In [174]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd, numpy as np
#from fastai.nlp import *
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pathlib import Path
import os, re, string

In [176]:
path = Path('C:/Users/xbabu/Babu/pers/PhD/MIST.7060 Data Analytics/Project/data/IMDB')
names = ['neg', 'pos']
# path = Path('../data/Toxic')
names = ['neg', 'pos']
os.listdir(path)

['test', 'train']

In [177]:
#os.listdir(path/'train/pos')[0:20]
#

In [178]:

def texts_from_files(src, names):
    texts,labels = [],[]
    for idx,name in enumerate(names):
        path = os.path.join(src, name)
        t = [o.strip() for o in open(path, encoding = "ISO-8859-1")]
        texts += t
        labels += ([idx] * len(t))
    return texts,np.array(labels)

def texts_from_folders(src, names):
    texts,labels = [],[]
    MAX_DOCS = 10000
    for idx,name in enumerate(names):
        path = os.path.join(src, name)
        i = 1
        for fname in sorted(os.listdir(path)):
            fpath = os.path.join(path, fname)
            texts.append(open(fpath, encoding="utf8").read())
            labels.append(idx)
            i = i + 1
            if i > MAX_DOCS: 
                break
    return texts,np.array(labels)

#def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [179]:
from sklearn.datasets import load_files
train, train_y = texts_from_folders(path/'train',names)
val, val_y = texts_from_folders(path/'test',names)

#??load_files
#files = load_files('C:/Users/xbabu/Babu/pers/PhD/MIST.7060 Data Analytics/Project/data/IMDB/train', encoding='utf-8')

In [180]:
#import nltk
#nltk.download('punkt')

import string
from nltk.stem import WordNetLemmatizer

def lemma_tokenizer(corpus): # a method to lemmatize corpus
    corpus = ''.join([ch for ch in corpus if ch not in string.punctuation]) # remove punctuation
    tokens = nltk.word_tokenize(corpus)
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens]

stopwords = nltk.corpus.stopwords.words('english')
stopwords.remove('but')
stopwords.remove('no')
stopwords.remove('not')
#print(stopwords)

In [181]:
#files
from sklearn.feature_extraction.text import CountVectorizer
args = {"stem": False, "lemmatize": True}
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤€‘’])')
def re_tokenizer(s): return re_tok.sub(r' \1 ', s).lower().split()
#ctvec = CountVectorizer(ngram_range=(1,2), lowercase=True, tokenizer=re_tokenizer, stop_words=stopwords, min_df=4, max_df=0.8, strip_accents='unicode')
#ctvec = CountVectorizer(ngram_range=(1,2), tokenizer=re_tokenizer, stop_words='english', min_df=4, max_df=0.8, strip_accents='unicode', lowercase=False)
#ctvec = CountVectorizer(ngram_range=(1,3), tokenizer=lemma_tokenizer, stop_words='english', min_df=4, max_df=0.8, strip_accents='unicode', lowercase=False)
ctvec = CountVectorizer(ngram_range=(1,1), tokenizer=lemma_tokenizer, stop_words=stopwords, min_df=4, max_df=0.8, strip_accents='unicode', lowercase=True)
#ctvec = CountVectorizer(tokenizer=lambda text: tokenizer(text, **args), stop_words='english', strip_accents='ascii', min_df=0, max_df=1., vocabulary=None)
#ctvec = CountVectorizer(tokenizer=tokenize)


In [182]:
trn_term_doc = ctvec.fit_transform(train)
val_term_doc = ctvec.transform(val)


In [183]:
#print(trn_term_doc)
trn_term_doc_dense = trn_term_doc.toarray()
trn_term_doc
#trn_term_doc_dense

<20000x27202 sparse matrix of type '<class 'numpy.int64'>'
	with 1956250 stored elements in Compressed Sparse Row format>

In [184]:
dictionary = ctvec.get_feature_names()
#dictionary
#pd.DataFrame(trn_term_doc_dense, columns=dictionary)

In [185]:
#w0 = set([o.lower() for o in train[1].split(' ')])
#w0

In [186]:
train_y

array([0, 0, 0, 0, ..., 1, 1, 1, 1])

In [187]:
#print(len(w0))
print(f'Training set size: {len(train)}')
print(f'Test set size: {len(val)}')
#print(ctvec.vocabulary_['writers'])
#trn_term_doc[1, 934]

Training set size: 20000
Test set size: 20000


## Naive Bayes

We define the **log-count ratio** $r$ for each word $f$:

$r = \log \frac{\text{ratio of feature $f$ in positive documents}}{\text ratio of feature $f$ in negative documents}}$

where ratio of feature $f$ in positive documents is the number of times a positive has a feature divided by the number of positive documents.


In [188]:
from time import time
t0 = time()
x = trn_term_doc
y = train_y

# grab the rows where the dependent varaible is 1
p = x[y==1].sum(0)+1
# grab the rows where the dependent varaible is 0
q = x[y==0].sum(0)+1
r = np.log((p/p.sum())/(q/q.sum()))
b = np.log(len(p)/len(q))
pre_preds = val_term_doc @r.T + b
preds = pre_preds.T>0
(preds == val_y).mean()

from sklearn import metrics
from sklearn.metrics import confusion_matrix
print('Results with BOW NB matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds.T))
cm = pd.DataFrame(confusion_matrix(val_y, preds.T, labels=[1,0]),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))

print("Elapsed Time: %0.3f seconds." % (time() - t0))

Results with BOW NB matrix:
Accuracy: 0.82695
Confusion matrix:
                  predicted positive  predicted negative
actual positive                7654                2346
actual negative                1115                8885
precision = 0.8728475310753792 , recall = 0.7654 , F1-score = 0.8156001918056369
Elapsed Time: 0.075 seconds.


### Binarized Naive Bayes

In [189]:
# Turns out that the version where we are actually looking at
# how often a word appears like "absurd" appeared twice. It turns out
# at least for this problem and quite often it doesn't matter
# whether "absurd" appeared twice or once
# all that matters is it appeared so what we tend to d here is
# take the term document matrix and do doc sign.
# doc.sign replaces anything positive as 1 and 
# anything negative with -1 so it binarizes it
# it gives a better result
t0 = time()
pre_preds = val_term_doc.sign() @r.T + b
#preds
preds = pre_preds.T>0
(preds == val_y).mean()

from sklearn import metrics
from sklearn.metrics import confusion_matrix
print('Results with BOW NB.sign() matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds.T))
cm = pd.DataFrame(confusion_matrix(val_y, preds.T, labels=[1,0]),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))
print("Elapsed Time: %0.3f seconds." % (time() - t0))

Results with BOW NB.sign() matrix:
Accuracy: 0.83485
Confusion matrix:
                  predicted positive  predicted negative
actual positive                7810                2190
actual negative                1113                8887
precision = 0.8752661660876387 , recall = 0.781 , F1-score = 0.8254505099614227
Elapsed Time: 0.063 seconds.


## Logistic Regression
instead of using the coefficients r.T, let us learn them using Logistic Regression

In [190]:
# In theory NB sounds ok, but rather than assuming that
# we should use these coefficients r.T why don't we learn them
# let su create a logistic regression
m = LogisticRegression(C=0.1, dual=False)
t0 = time()
m.fit(x,y)
preds = m.predict(val_term_doc)
(preds==val_y).mean()

from sklearn import metrics
from sklearn.metrics import confusion_matrix
print('Results with BOW LR :\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds))
cm = pd.DataFrame(confusion_matrix(val_y, preds, labels=[1,0]),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))
print("Elapsed Time: %0.3f seconds." % (time() - t0))

Results with BOW LR :
Accuracy: 0.87565
Confusion matrix:
                  predicted positive  predicted negative
actual positive                8704                1296
actual negative                1191                8809
precision = 0.8796361798888327 , recall = 0.8704 , F1-score = 0.8749937170143252
Elapsed Time: 1.010 seconds.


### Binarized LR

In [191]:
t0 = time()
m = LogisticRegression(C=0.1, dual=False)
m.fit(trn_term_doc.sign(),y)
preds = m.predict(val_term_doc.sign())
(preds==val_y).mean()

from sklearn import metrics
from sklearn.metrics import confusion_matrix
print('Results with BOW LR.sign() matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds))
cm = pd.DataFrame(confusion_matrix(val_y, preds, labels=[1,0]),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))
print("Elapsed Time: %0.3f seconds." % (time() - t0))

Results with BOW LR.sign() matrix:
Accuracy: 0.87655
Confusion matrix:
                  predicted positive  predicted negative
actual positive                8767                1233
actual negative                1236                8764
precision = 0.8764370688793361 , recall = 0.8767 , F1-score = 0.8765685147227916
Elapsed Time: 0.853 seconds.


## Support Vector Machine

In [192]:
from time import time
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
t0 = time()
svc = SVC(kernel='linear', C=1)
svc.fit(x,y)
#print("Elapsed Time: %0.3f seconds." % (time() - t0))
#t0 = time()
preds = svc.predict(val_term_doc)
print('Results with BOW SVM matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds))
#print("Elapsed Time: %0.3f seconds." % (time() - t0))
from sklearn import metrics
from sklearn.metrics import confusion_matrix
print('Results with BOW SVM matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds))
cm = pd.DataFrame(confusion_matrix(val_y, preds, labels=[1,0]),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))
print("Elapsed Time: %0.3f seconds." % (time() - t0))

Results with BOW SVM matrix:
Accuracy: 0.83655
Results with BOW SVM matrix:
Accuracy: 0.83655
Confusion matrix:
                  predicted positive  predicted negative
actual positive                8223                1777
actual negative                1492                8508
precision = 0.8464230571281524 , recall = 0.8223 , F1-score = 0.8341871671316257
Elapsed Time: 631.957 seconds.


### Binarized SVM

In [193]:
from time import time
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
t0 = time()
svc = SVC(kernel='linear', C=1)
svc.fit(trn_term_doc.sign(),y)
#print("Elapsed Time: %0.3f seconds." % (time() - t0))
#t0 = time()
preds = svc.predict(val_term_doc.sign())
print('Results with BOW SVM.sign() matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds))
#print("Elapsed Time: %0.3f seconds." % (time() - t0))
from sklearn import metrics
from sklearn.metrics import confusion_matrix
print('Results with BOW SVM.sign() matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds))
cm = pd.DataFrame(confusion_matrix(val_y, preds, labels=[1,0]),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))
print("Elapsed Time: %0.3f seconds." % (time() - t0))


Results with BOW SVM.sign() matrix:
Accuracy: 0.84345
Results with BOW SVM.sign() matrix:
Accuracy: 0.84345
Confusion matrix:
                  predicted positive  predicted negative
actual positive                8329                1671
actual negative                1460                8540
precision = 0.8508529982633568 , recall = 0.8329 , F1-score = 0.8417807873060791
Elapsed Time: 622.871 seconds.


In [194]:
x = trn_term_doc.sign()
y = train_y
# x[y==1].sum(0) is to calculate the ratio of each feature in positive and no negative by dividing the toekn counts.

val_x = val_term_doc.sign()
p = x[y==1].sum(0)+1
q = x[y==0].sum(0)+1
r = np.log((p/p.sum())/(q/q.sum()))
b = np.log(len(p)/len(q))

## NB-LR

In [195]:
################ NB-LR #############################
# take the term-document matrix and multiply by r
# every where 0 appears, 0 appears in x_nb and 
# every where 1 appears in TD, the r appears there
t0 = time()
x_nb = x.multiply(r)
# we are going to use x_nb as independent variables instead in the
# logistic regression
m = LogisticRegression(dual=False, C=0.1)
m.fit(x_nb, y)

# do the samething for the validation set and get the predictions
# to get a better result
val_x_nb = val_x.multiply(r)
preds = m.predict(val_x_nb)
#(preds.T==val_y).mean()
print('Results with BOW NB-LR matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds.T))
cm = pd.DataFrame(confusion_matrix(val_y, preds.T, labels=[1,0]),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))
print("Elapsed Time: %0.3f seconds." % (time() - t0))

Results with BOW NB-LR matrix:
Accuracy: 0.88525
Confusion matrix:
                  predicted positive  predicted negative
actual positive                8867                1133
actual negative                1162                8838
precision = 0.884136005583807 , recall = 0.8867 , F1-score = 0.8854161465874482
Elapsed Time: 0.677 seconds.


### Binarized NB-LR

In [196]:
################ NB-LR Sign() #############################
# take the term-document matrix and multiply by r
# every where 0 appears, 0 appears in x_nb and 
# every where 1 appears in TD, the r appears there
t0 = time()
x_nb = x.sign().multiply(r)
# we are going to use x_nb as independent variables instead in the
# logistic regression
m = LogisticRegression(dual=False, C=0.1)
m.fit(x_nb, y)

# do the samething for the validation set and get the predictions
# to get a better result
val_x_nb = val_x.sign().multiply(r)
preds = m.predict(val_x_nb)
#(preds.T==val_y).mean()
print('Results with BOW NB-LR matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds.T))
cm = pd.DataFrame(confusion_matrix(val_y, preds.T, labels=[1,0]),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))
print("Elapsed Time: %0.3f seconds." % (time() - t0))

Results with BOW NB-LR matrix:
Accuracy: 0.88525
Confusion matrix:
                  predicted positive  predicted negative
actual positive                8867                1133
actual negative                1162                8838
precision = 0.884136005583807 , recall = 0.8867 , F1-score = 0.8854161465874482
Elapsed Time: 0.720 seconds.


## NB-SVM

In [197]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
t0 = time()
svc = SVC(kernel='linear', C=1e5)
svc.fit(x_nb, y)
print("Elapsed Time: %0.3f seconds." % (time() - t0))

t0 = time()
preds = svc.predict(val_x_nb)
print('Results with BOW NB-SVM matrix:\n' + 'Accuracy:', metrics.accuracy_score(val_y, preds.T))
cm = pd.DataFrame(confusion_matrix(val_y, preds, labels=[1,0]),
                  index=['actual positive', 'actual negative'],
                  columns=['predicted positive', 'predicted negative'])
print('Confusion matrix:\n', cm)
TP = cm.at['actual positive','predicted positive']
FP = cm.at['actual negative','predicted positive']
FN = cm.at['actual positive','predicted negative']
print('precision =', TP/(TP+FP), ', recall =', TP/(TP+FN), ', F1-score =', 2*TP/(2*TP+FP+FN))
print("Elapsed Time: %0.3f seconds." % (time() - t0))

Elapsed Time: 394.508 seconds.
Results with BOW NB-SVM matrix:
Accuracy: 0.83785
Confusion matrix:
                  predicted positive  predicted negative
actual positive                8172                1828
actual negative                1415                8585
precision = 0.8524042974861792 , recall = 0.8172 , F1-score = 0.8344310001531628
Elapsed Time: 73.786 seconds.
